In [1]:
from LoadDataModule import *
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools    
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from math import ceil

In [2]:
# Part 1: ANN classifier
# Load images and labels using LoadDataModule()
ld = LoadDataModule()
images, labels = ld.load('train')
labels = labels.reshape(-1,1)

from sklearn.preprocessing import MultiLabelBinarizer
#y_true = to_categorical(data)
enc = OneHotEncoder()
enc.fit(labels)
y_true = enc.transform(labels).toarray()
y_true

shape = (-1, 1, 28, 28)
from sklearn.model_selection import train_test_split
XTrain, XValid, yTrain, yValid = train_test_split(images, y_true, test_size=0.2, shuffle=False)
XTrain = XTrain.reshape(shape)
XValid = XValid.reshape(shape)
XTrain.shape


(48000, 1, 28, 28)

In [3]:
# From sklearn's website
# Located here: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [4]:
from sklearn.preprocessing import MultiLabelBinarizer
#y_true = to_categorical(data)
enc = OneHotEncoder()
enc.fit(labels)
y_true = enc.transform(labels).toarray()
y_true.shape

(60000, 10)

In [5]:
test_imgs, test_labels = ld.load('test')
test_labels = test_labels.reshape(-1,1)
enc.fit(test_labels)
test_labels = enc.transform(test_labels).toarray()
test_labels.shape

(10000, 10)

In [6]:
class_names = ['T-shirt / top', 
              'Trouser',
              'Pullover',
              'Dress',
              'Coat',
              'Sandal',
              'Shirt',
              'Sneaker',
              'Bag',
              'Ankle boot']

In [83]:
# VECTORIZE
def get_im2col_indices(x_shape, field_height=3, field_width=3, padding=1, stride=1, kc=1):
    # First figure out what the size of the output should be
    N, H, W, C = x_shape
    #print("N = ", N)
    #print("H = ", H)
    #print("W = ", W)
    #print("C = ", C)
    if (H + 2 * padding - field_height) % stride != 0:
        H-=1
        W-=1
    
    assert (H + 2 * padding - field_height) % stride == 0
    assert (W + 2 * padding - field_height) % stride == 0
    out_height = (H + 2 * padding - field_height) / stride + 1
    #print("out_height = ", out_height)
    out_width = (W + 2 * padding - field_width) / stride + 1
    #print("out_width = ", out_width)
    
    #print("kc = ", kc)
    i0 = np.repeat(np.arange(field_height, dtype='int32'), field_width)
    #print("i0=", i0.shape)
    i0 = np.tile(i0, C)
    #i0 = np.tile(i0, kc)
    #print("i0 = ", i0.shape)
    i1 = stride * np.repeat(np.arange(out_height ,dtype='int32'), out_width)
    #print("i1 = ", i1.shape)
    j0 = np.tile(np.arange(field_width), field_height * C)
    #j0 = np.tile(np.arange(field_width), field_height * kc)
    #print("j0 = ", j0.shape)
    j1 = stride * np.tile(np.arange(out_width,dtype='int32'), int(out_height))

    i = i0.reshape(-1, 1) + i1.reshape(1, -1)
    j = j0.reshape(-1, 1) + j1.reshape(1, -1)

    k = np.repeat(np.arange(C,dtype='int32'), field_height * field_width).reshape(-1, 1)
    #print("k = ", k.shape)

    return (k, i, j)

def im2col_indices(x, field_height=3, field_width=3, padding=1, stride=1, kc=1):
    """ An implementation of im2col based on some fancy indexing """
    # Zero-pad the input
    p = padding
    #print("padding = ", p)
    x_padded = np.pad(x, ((0, 0), (p, p), (p, p), (0, 0)), mode='constant')
    #print("x_padded.shape = ", x_padded.shape)

    k, i, j = get_im2col_indices(x.shape, field_height, field_width, padding, stride, kc=kc)
    #print("k.shape = ", k.shape)
    #print("i.shape = ", i.shape)
    #print("j.shape = ", j.shape)

    cols = x_padded[:, i, j, k]
    #print("x.shape = ", x.shape)
    #C = x.shape[3]
    C = kc
    #print("C = ", C)
    cols = cols.transpose(1, 2, 0).reshape(field_height * field_width * C, -1)
    #print("cols.shape = ", cols.shape, " end of im2col back to conv\n")
    return cols


def col2im_indices(cols, x_shape, field_height=3, field_width=3, padding=1, stride=1):
    """ An implementation of col2im based on fancy indexing and np.add.at """
    N, H, W, C = x_shape
    #print("N = ", N)
    #print("H = ", H)
    #print("W = ", W)
    #print("C = ", C)
    H_padded, W_padded = H + 2 * padding, W + 2 * padding
    x_padded = np.zeros((N, C, H_padded, W_padded), dtype=cols.dtype)
    k, i, j = get_im2col_indices(x_shape, field_height, field_width, padding,
                               stride)
    cols_reshaped = cols.reshape(C * field_height * field_width, -1, N)
    #print("cols_reshaped = ", cols_reshaped.shape)
    cols_reshaped = cols_reshaped.transpose(2, 0, 1)
    np.add.at(x_padded, (slice(None), k, i, j), cols_reshaped)
    if padding == 0:
        return x_padded
    return x_padded[:, :, padding:-padding, padding:-padding]


In [148]:
# LAYERS
class Conv():
    def __init__(self, X_dim, kernel_w, kernel_h, kernel_c, F_out, stride, padding):

        _, self.H, self.W, self.C = X_dim
        print("Conv Object is getting made\n")
        #print("self.N = ", self.N)
        print("self.H = ", self.H)
        print("self.W = ", self.W)
        print("self.C = ", self.C)

        self.F_out = F_out
        #print("self.F_out = ", F_out)

        self.kh, self.kw, self.kc = kernel_h, kernel_w, kernel_c
        self.stride, self.padding = stride, padding

        self.weights = np.random.randn(self.F_out, kernel_h, kernel_w, kernel_c) / np.sqrt(self.kc / 2.)
        self.b = np.zeros((self.F_out,1))
        #self.b = np.zeros((self.kc,1))
        self.params = [self.weights, self.b]
        #print(self.weights)

        self.h_out = (self.H - kernel_h + 2 * padding) // stride + 1
        self.w_out = (self.W - kernel_w + 2 * padding) // stride + 1
        #print("self.h_out = ", self.h_out)
        #print("self.w_out = ", self.w_out)
        
        
        #if not self.h_out.is_integer() or not self.w_out.is_integer():
        #    raise Exception("Invalid dimensions!")

        #self.h_out, self.w_out = self.h_out, self.w_out
        self.out_dim = (self.kc, self.h_out, self.w_out)
        
    def toString(self):
        
        print("\nOther hyperparams:\n X_dim(i.e., channels) = (", str(self.N),
              ", ", str(self.C), ", ", str(self.H), ", ", str(self.W), ")\nNum Filters: ", 
              str(self.kc), "\nFilter Height: ", str(self.kh), "\nFilter Width: ", 
              str(self.kw), "\nStride: ", str(self.stride), "\nPadding: ", str(self.padding), 
              "\nW.shape[0]: ", str(self.weights.shape[0]), "\nW.shape[1]: ", str(self.weights.shape[1]), 
              "\nW.shape[2]: ", str(self.weights.shape[2]), "\nb.shape[0]: ",
              str(self.b.shape[0]), "\nb.shape[1]: ", str(self.b.shape[1]), "\nout_dim[0]: ",
              str(self.out_dim[0]), "\nout_dim[1]: ", str(self.out_dim[1]), "\nout_dim[2]: ", str(self.out_dim[2]))
    
    def forward(self, x):
        '''
        Inputs:
        x = Input data of shape (N, H, W, C)
        self.W = Filter weights of shape (F, kh, kw, kc)
        self.b = Biases of shape (F,)
        self.stride = stride
        self.padding = padding
    
        Outputs:
        out = output data of shape (N, F, H', W')
        H' = 1 + (H + 2*pad - kh) / stride
        W' = 1 + (W + 2*pad - kw) / stride
        cache = (x, w, b, conv_param)
        '''
    
        #print("Start of Conv Forward\n")
        out = None
        pad_num = self.padding
        stride = self.stride
        F = self.F_out
        #print("F = ", F)
        
        N, H, W, C = x.shape
        self.N = N
        #print("N = ", N)
        #print("H = ", H)
        #print("W = ", W)
        #print("C = ", C)
        
        H_prime = 1 + (H + 2*pad_num - self.kh) / stride
        W_prime = 1 + (W + 2*pad_num - self.kw) / stride
        
        x_reshaped = x.reshape(x.shape[0]*x.shape[3], x.shape[1], x.shape[2], 1)

        #print("To im2col_indices\n")
        self.x_col = im2col_indices(x_reshaped, self.kh, self.kw, stride=self.stride, padding=self.padding, kc=self.kc)
        #print("Back from im2col_indices\n")
        #print("x_col.shape = ", self.x_col.shape)
        
        w_row = self.weights.reshape(F, -1)
        #print("w_row.shape = ", w_row.shape)
        
        out = w_row @ self.x_col + self.b
        #print("out shape before reshaping = ", out.shape)
        
        out = out.reshape(N, self.h_out, self.w_out, F) 
        #print("out shape after reshaping to 2D = ", out.shape, "\nend of Conv.forward()\n\n")

        
        return out

    def backward(self, dout):
        '''
        Inputs: 
        dout = upstream derivatives
        cache = a tuple of (x, w, b, conv_param)
    
        Output:
        dx = gradient with respect to x
        dw = gradient with respect to w
        db = gradient with respect to b
        '''
    
        dout_flat = dout.transpose(3, 1, 2, 0).reshape(self.F_out, -1)
        dw = dout_flat @ self.x_col.T
        dw = dw.reshape(self.weights.shape)
        #print("dw.shape = ", dw.shape)
        #db = np.sum(dout, axis=(1,2))
        #print(db.shape)
        db = np.sum(dout, axis=(0, 1, 2)).reshape(self.F_out,-1)
        #print("db.shape = ", db.shape)
        
        w_flat = self.weights.reshape(self.F_out, -1)
        dx_col = w_flat.T @ dout_flat
        shape = (self.N, self.H, self.W, self.C)
        #print("Conv backward shape = ", shape)
        dx = col2im_indices(dx_col, shape, self.kh, self.kw, self.padding, self.stride)
        
        return dx, [dw, db]
    

class Maxpool():
    def __init__(self, X_dim, size, stride):
        _, self.H, self.W, self.C = X_dim
        self.params = []

        self.size = size
        self.stride = stride
                          
        self.h_out = (self.H - size) // stride + 1
        #print("self.h_out = ", self.h_out)
        self.w_out = (self.W - size) // stride + 1
        #print("self.w_out = ", self.h_out)


        self.h_out, self.w_out = int(self.h_out), int(self.w_out)
        self.out_dim = (self.C, self.h_out, self.w_out)
                          
    def forward(self, x):
        '''
        Inputs: 
        x = 4D tensor of shape (N, H, W, C)
        self.size
    
        Output:
        out = pooled output
        cache = input x and pool_param
        '''
        self.x = x
        N, H, W, C = x.shape
        self.N = N
        #print("N = ", N)
        #print("H = ", H)
        #print("W = ", W)
        #print("C = ", C)
        
        stride = self.stride
        x_reshaped = x.reshape(x.shape[0]*x.shape[3], x.shape[1], x.shape[2], 1)
        self.x_col = im2col_indices(x_reshaped, self.size, self.size, padding=0, stride=self.stride)
        #print("self.x_col.shape = ", self.x_col.shape)
        
        self.max_indices = np.argmax(self.x_col, axis=0)
        out = self.x_col[self.max_indices, range(self.max_indices.size)]
        
        out = out.reshape(self.N, self.h_out, self.w_out, self.C).transpose(1,2,0,3)
        out = out.reshape(out.shape[2], out.shape[0], out.shape[1], out.shape[3])
        #print(out.shape)
         
        return out

    def backward(self, dout):
        '''
        Inputs:
        dout = upstream derivatives, same size as  cached x
        cache = tuple of (x, pool_param)

        Output:
        dx = gradient with respect to x
        '''
        
        dx_col = np.zeros_like(self.x_col)
        dout_flat = dout.transpose(2,0,1,3).ravel()
        
        dx_col[self.max_indices, range(self.max_indices.size)] = dout_flat
        
        shape = (self.N, self.H, self.W, self.C)
        #print("maxpool backward shape = ", shape)
        dx = col2im_indices(dx_col, shape, self.size, self.size, padding=0, stride=self.stride)
        dx = dx.reshape(self.N, self.H, self.W, self.C)

        return dx, []

    
class relu():
    def __init__(self):
        self.params = []
        
    def forward(self, z):
        '''
        Input:
        z = input of any shape

        Output:
        tuple of (out, cache)
        '''
        self.z = z
        relu = lambda z: z * (z > 0)
        out = relu(z)

        return out

    def backward(self, dout):
        '''
        Input:
        dout = post-activation gradient
        cache = previous x used in forward prop 

        Output:
        dz = gradient with respect to z
        '''

        dz, z = None, self.z
        dz = dout * (z > 0)

        assert (dz.shape == z.shape)

        return dz, []
    

class sigmoid():
    def __init__(self):
        self.params = []
        
    def forward(self, z):
        '''
        Input: 
        z = input of any shape

        Output:
        tuple of (out, cache)
        '''
        out = 1 / (1 + np.exp(-z))
        self.z = z

        return out

    def backward(self, dout):
        '''
        Input:
        dout = post-activation gradient
        self.z = z

        Output:
        dz = gradient with respect to z
        '''

        z = self.z
        s = 1 / (1 + np.exp(-z))
        dz = dout * s * (1 - s)

        assert (dz.shape == z.shape)

        return dz, []

class tanh():
    def __init__(self):
        self.params = []
        
    def forward(self, z):
        '''
        Input:
        z = input of any shape

        Output:
        tuple of (out, cache)
        '''
        self.out = np.tanh(z)
        self.z = z
        return self.out

    def backward(self, dout):
        '''
        Input:
        dout = post-activation gradient
        self.z = z

        Output:
        dz = gradient with respect to z
        '''
        z = self.z
        dz = dout * (1 - np.square(self.out))

        return dz, []

    
class fully_connected():
    def __init__(self, in_size, out_size):
        self.W = np.random.randn(in_size, out_size) / np.sqrt(in_size / 2.)
        self.b = np.zeros((1, out_size))
        self.params = [self.W, self.b]
        
    def forward(self, x):
        '''
        Inputs:
        x = input (batch size, d_1, ..., d_k)
        self.W = weights of shape (input dim, # of outputs of fc layer)
        self.b = bias (# of outputs of fc layer,)

        Output:
        out = output of shape (batch size, # of outputs of fc layer)
        cache = x
        '''

        N = x.shape[0]

        # 2D matrix [N, D]
        reshape_input = x.reshape(N, -1)

        out = np.dot(x,self.W) + self.b

        self.x = x
        return out
    

    def backward(self, dout):
        '''
        Inputs:
        dout = partial derivative with respecty to loss of shape (N, M)
        cache = (x,w,b)

        Output:
        dx = gradient with respect to x, of shape (N, d_1,...,d_k)
        dw = gradient with respect to w, of shape (input dim, output of fc-layer)
        db = gradient with respect to b, of shape (output of fc-layer,)
        '''

        x = self.x
        w = self.W
        b = self.b
        
        N = x.shape[0]
        # dX
        dx = np.dot(dout, w.T)
        dx = dx.reshape(x.shape)

        # dW
        reshape_input = x.reshape(N, -1)
        dw = reshape_input.T.dot(dout)

        # dB
        db = reshape_input.T.dot(dout)

        db = np.sum(dout, axis=0)

        return dx, [dw, db]

    
class flatten():
    def __init__(self):
        self.params = []
        
    def forward(self, x):
        out_shape = (x.shape[0], -1)
        out = x.ravel().reshape(out_shape)
        out_shape = out_shape[1]
        self.x = x
        return out

    def backward(self, dout):
        x = self.x
        out = dout.reshape(x.shape)
        return out, ()

In [149]:
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

In [150]:
def crossEntropyLoss(X, y):
    #X is the output from fully connected layer (num_examples x num_classes)
    #y is labels (num_examples x 1)
    m = y.shape[0]
    p = softmax(X)
    log_likelihood = -np.log(p[range(m), y])
    loss = np.sum(log_likelihood) / m

    #and its derivative
    dx = p.copy()
    dx[range(m), y] = dx[range(m), y] - 1
    dx = dx / m
    
    return loss, dx

In [151]:
class CNN:
    def __init__(self, layers, loss_func=crossEntropyLoss):
        self.layers = layers
        self.params = []
        for layer in self.layers:
            self.params.append(layer.params)
        self.loss_func = loss_func

    def forward(self, X):
        for layer in self.layers:
            X = layer.forward(X)
        return X

    def backward(self, dout):
        grads = []
        for layer in reversed(self.layers):
            dout, grad = layer.backward(dout)
            grads.append(grad)
        return grads

    def train_step(self, X, y):
        out = self.forward(X)
        loss, dout = self.loss_func(out, y)
        #loss += l2_regularization(self.layers)
        grads = self.backward(dout)
        #grads = delta_l2_regularization(self.layers, grads)
        return loss, grads

    def predict(self, X):
        X = self.forward(X)
        return np.argmax(softmax(X), axis=1)


In [182]:
# OPTIMIZER
def accuracy(y_true, y_pred):
    return np.mean(y_pred == y_true)  # both are not one hot encoded

def minibatch(x, y, batch_size):
    m = x.shape[0]
    mini_batches = []
    for i in range(0, m, batch_size):
        x_batch = x[i:i+batch_size, :, :, :]
        #print(x_batch.shape)
        y_batch = y[i:i+batch_size, :]
        mini_batches.append((x_batch, y_batch))
    return mini_batches

def vanilla_update(params, grads, learning_rate=0.001):
    for param, grad in zip(params, reversed(grads)):
        for i in range(len(grad)):
            #print("i = ", i)
            #print("param[i] = ", param[i])
            #print("param[i].shape = ", param[i].shape)
            #print("grad[i] = ", grad[i])
            #print("grad[i].shape = ", grad[i].shape)
            param[i] += -learning_rate*grad[i]
            
def sgd(nnet, xTrain, yTrain, batch_size, epoch, learning_rate, verbose=True, xTest=None, yTest=None):
    minibatches = minibatch(xTrain, yTrain, batch_size)
    for i in range(epoch):
        loss = 0
        if verbose:
            print('Epoch {0}'.format(i+1))
            for x_mini, y_mini in minibatches:
                loss, grads = nnet.train_step(x_mini, y_mini)
                vanilla_update(nnet.params, grads, learning_rate=learning_rate)
            if verbose:
                train_acc = accuracy(yTrain, nnet.predict(xTrain))
                test_acc = accuracy(yTest, nnet.predict(xTest))
                print('Loss = {0} | Training Accuracy = {1} | Test Accuracy {2}'.format(loss, train_acc, test_acc))
    return nnet

In [195]:
# Set random seed to Student ID
seed = 105792018
np.random.seed(seed)

# Split data into train and validation sets
# images_train 48000, 784
# yval_train 48000, 1

images_train, images_test, labels_train, labels_test = train_test_split(images,labels,test_size=0.2,shuffle=False)

img_rows, img_cols = 28, 28

images_train = np.reshape(images_train, (-1, img_rows, img_cols, 1))
images_train = images_train.astype(np.float32)/255
#print(images_train.shape)
images_test = np.reshape(images_test, (-1, img_rows, img_cols, 1))
images_test = images_test.astype(np.float32)/255
X_dims = images_train.shape
print(images_train.shape)
print(labels_train.shape)
print(images_test.shape)
print(labels_test.shape)

(48000, 28, 28, 1)
(48000, 1)
(12000, 28, 28, 1)
(12000, 1)


In [196]:
X_dims = (200, 28, 28, 1)
conv1 = Conv(X_dims, kernel_w=3, kernel_h=3, kernel_c=1, F_out= 2, stride=1, padding=0)
#self, X_dim, kernel_n, kernel_w, kernel_h, kernel_c, stride, padding
#conv1.toString()

Conv Object is getting made

self.H =  28
self.W =  28
self.C =  1


In [197]:
#print("XTrain shape = ", XTrain.shape)
#X_Train = XTrain.reshape(XTrain.shape[0], XTrain.shape[2], XTrain.shape[3], XTrain.shape[1])
#print("X_Train shape = ", X_Train.shape)
#out = conv1.forward(X_Train)
#out.shape

In [198]:
#X_dim, size, stride
#print("out.shape = ", out.shape)
#pool1 = Maxpool(out.shape, 2,2)
pool1 = Maxpool((200,26,26,2), 2, 2)
#out2 = pool1.forward(out)
#out2.shape

In [199]:
#self, X_dim, kernel_w, kernel_h, kernel_c, F, stride, padding

conv2 = Conv((200,13,13,2), 3, 3, 2, 2, 1, 0)
#conv2 = Conv(out2.shape, 3,3,2,2,1,0)
#out3 = conv2.forward(out2)
#out3.shape

Conv Object is getting made

self.H =  13
self.W =  13
self.C =  2


In [200]:
pool2 = Maxpool((200,11,11,2), 2, 2)
#pool2 = Maxpool(out3.shape, 2, 2)
#out4 = pool2.forward(out3)

In [201]:
#out4.shape
flat = flatten()
#flat1 = flat.forward()
#flat1.shape

In [202]:
fc = fully_connected(50, 10)
#fc1 = fc.forward(flat1)
#fc1.shape

In [208]:
sig = sigmoid()
hyptanh = tanh()
hyptanh2 = tanh()
re = relu()

In [209]:
cnn = CNN([conv1, hyptanh, pool1, conv2, re, pool2, flat, fc, hyptanh2])

In [211]:
#def sgd(nnet, xTrain, yTrain, batch_size, epoch, learning_rate, verbose=True, xTest=None, yTest=None):
cnn_train = sgd(cnn, images_train, labels_train, 200, 20, 0.01, True, xTest=images_test, 
                yTest=labels_test)

Epoch 1
Loss = 460.5170185988091 | Training Accuracy = 0.09925 | Test Accuracy 0.103
Epoch 2
Loss = 460.5170185988091 | Training Accuracy = 0.09925 | Test Accuracy 0.103
Epoch 3
Loss = 460.5170185988091 | Training Accuracy = 0.09925 | Test Accuracy 0.103
Epoch 4
Loss = 460.5170185988091 | Training Accuracy = 0.09925 | Test Accuracy 0.103
Epoch 5
Loss = 460.5170185988091 | Training Accuracy = 0.09925 | Test Accuracy 0.103
Epoch 6
Loss = 460.5170185988091 | Training Accuracy = 0.09925 | Test Accuracy 0.103
Epoch 7
Loss = 460.5170185988091 | Training Accuracy = 0.09925 | Test Accuracy 0.103
Epoch 8
Loss = 460.5170185988091 | Training Accuracy = 0.09925 | Test Accuracy 0.103
Epoch 9
Loss = 460.5170185988091 | Training Accuracy = 0.09925 | Test Accuracy 0.103
Epoch 10
Loss = 460.5170185988091 | Training Accuracy = 0.09925 | Test Accuracy 0.103
Epoch 11
Loss = 460.5170185988091 | Training Accuracy = 0.09925 | Test Accuracy 0.103
Epoch 12
Loss = 460.5170185988091 | Training Accuracy = 0.09925

In [168]:
cnn.predict(test_ims)

array([6, 6, 6, ..., 6, 6, 6], dtype=int64)

In [167]:
test_ims = XTrain.reshape(48000,28,28,1)